In [1]:
import pandas as pd
import numpy as np
import re
import torch
import json
from tqdm.notebook import tqdm
# from torch.utils.data import Dataset, DataLoader
# from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split
import ast

In [2]:
import sys

sys.path.append('../utils')
from evaluator import Evaluator
from json_format import process_json, unprocess_json

In [3]:
train = pd.read_csv('../data/distill_100k.csv', index_col=0, converters={'json': ast.literal_eval})
val_set = pd.read_csv('../data/val_set_300_sb_valid.csv', index_col=0, converters={'json': json.loads})
manual_test = pd.read_csv('../data/manual_test_100.csv', index_col=0, converters={'json': json.loads})

In [4]:
len(train), len(val_set), len(manual_test)

(100000, 500, 100)

In [5]:
train = train[train.Text.apply(lambda txt: txt not in val_set.Text.values and txt not in manual_test.Text.values)]
len(train)

99382

In [6]:
train = train.sample(n=50000, random_state=42)
len(train)

50000

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base")

In [8]:
tokenizer.add_tokens(["<BOB>", "<EOB>", "<BOT>", "<EOT>", "<BOP>", "<EOP>", "<BOC1>", "<EOC1>", "<BOC2>", "<EOC2>"])
model.resize_token_embeddings(len(tokenizer))


assert train.loc[train.index[0], 'json'] == unprocess_json(process_json(train.loc[train.index[0], 'json']))

In [9]:
from datasets import Dataset

ads_dataset = Dataset.from_pandas(train[["Text", "json"]])
ads_dataset = ads_dataset.train_test_split(test_size=0.005, seed=42)
ads_dataset = ads_dataset.flatten()
ads_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 49750
    })
    test: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 250
    })
})

In [10]:
def preprocess_function(examples):
    inputs = [text for text in examples["Text"]]
    # targets = ['' for bundles in examples["json"]]
    targets = [process_json(bundles) for bundles in examples["json"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

ads = ads_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ads_dataset["train"].column_names
)
ads = ads.flatten()

# ads_test = ads_test_dataset.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=ads_test_dataset.column_names
# )
# ads_test = ads_test.flatten()


Map (num_proc=4):   0%|          | 0/49750 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/250 [00:00<?, ? examples/s]

In [11]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [12]:
from transformers import DataCollatorForSeq2Seq

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [13]:
class MetricComputer:
  def __init__(self, batch_size=8):
    self.generations = []
    self.batch_size=batch_size

  def __call__(self, eval_preds):
    ev = Evaluator(val_set, model=model, tokenizer=tokenizer, batch_size=self.batch_size)
    stats = ev.calc_bleu_batched()
    self.generations.append(ev.generate_samples_batched(count=20))
    # clear_output()
    return stats

In [14]:
n_epochs = 5

training_args = Seq2SeqTrainingArguments(
    output_dir="ruT5-large",
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    # eval_steps=600,
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=n_epochs,
    # predict_with_generate=True,
    generation_max_length=128,
    fp16=True,
    lr_scheduler_type="cosine",
    group_by_length=False,
    warmup_steps=3,
)

mc = MetricComputer(batch_size=32)
empty_dataset = Dataset.from_dict({"Text": [], "json": []})
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ads["train"],
    eval_dataset=ads["test"],
    # eval_dataset=empty_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=mc,
)

In [15]:
trainer.train()

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bep-sb,Bep-multi,Ta-bleu-sb,Ta-chrf-sb,Ta-chrf-multi,Eb-ind,Mb-ind,Bleu-classic,Chrf-classic,Chrf-classic-multi,Bleu Old,Failed Ratio
1,1.122200,0.207195,0.688990,0.627768,39.510144,78.573960,76.734202,0.072000,0.066000,50.388247,75.550502,74.613057,50.388247,0.002000
2,0.266400,0.168281,0.699075,0.658481,40.462668,78.427558,77.385780,0.034000,0.058000,48.082766,75.377632,74.928967,48.082766,0.004000
3,0.226600,0.152546,0.706601,0.672972,40.768771,79.472847,79.252912,0.032000,0.056000,49.512203,76.434079,77.236636,49.512203,0.000000
4,0.196200,0.142888,0.710614,0.669965,41.712719,79.393518,78.618689,0.044000,0.056000,52.030425,76.639484,76.636524,52.030425,0.002000
5,0.187600,0.142500,0.708689,0.667081,41.472250,79.354457,78.976409,0.046000,0.058000,51.156401,76.517354,77.032086,51.156401,0.000000


<BOT> Sigma 30 1.4 +600<EOP><BOC1> 1<EOC1><BOC2> GEL<EOC2>
<BOT> Sigma 30 1.4 +600<EOP><BOC1> 1<EOC1><BOC2> GEL<EOC2>
<BOT> Сингл и 130<EOP><BOC1> 1<EOC1><BOC2> EUR<EOC2>
<BOT> Наволочки<EOT><BOP> 2 шт<EOC1><BOC2> RUB<EOC2>


TrainOutput(global_step=3890, training_loss=0.34482914353397637, metrics={'train_runtime': 2461.1188, 'train_samples_per_second': 101.072, 'train_steps_per_second': 1.581, 'total_flos': 3.78678918062592e+16, 'train_loss': 0.34482914353397637, 'epoch': 5.0})

In [18]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
output_dir = "ruT5-base-trained-distill-data"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('ruT5-base-trained-distill-data/tokenizer_config.json',
 'ruT5-base-trained-distill-data/special_tokens_map.json',
 'ruT5-base-trained-distill-data/spiece.model',
 'ruT5-base-trained-distill-data/added_tokens.json',
 'ruT5-base-trained-distill-data/tokenizer.json')

: 